![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will build a model on customer churn binary classification problem. You will be using `files_for_lab/Customer-Churn.csv` file.

**Scenario**

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

**Instructions**

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.
- Read that data into Python and call the dataframe `churnData`.
- Check the datatypes of all the columns in the data. You would see that the column `TotalCharges` is object type. Convert this column into numeric type using `pd.to_numeric` function.
- Check for null values in the dataframe. Replace the null values.
- Use the following features: `tenure`, `SeniorCitizen`, `MonthlyCharges` and `TotalCharges`:
  - Scale the features either by using normalizer or a standard scaler.
  - Split the data into a training set and a test set.
  - Fit a logistic regression model on the training data.
  - Check the accuracy on the test data.

**Note**: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model is.

## Imports

In [127]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

## Load & clean data

In [128]:
churnData = pd.read_csv('files_for_lab/Customer-Churn.csv')
churnData.columns =churnData.columns.str.lower().str.replace(' ', '_').str.replace('.', '', regex=True)

In [129]:
churnData.columns

Index(['gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [130]:
churnData.dtypes

gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

i = 0
for value in churnData['totalcharges']:
    if value == ' ':
        churnData['totalcharges'].iloc[i] = ''
    i = i+1

In [131]:
churnData['totalcharges'] = pd.to_numeric(churnData['totalcharges'], errors='coerce')

In [132]:
churnData['totalcharges'].isnull().sum()

11

In [133]:
churnData['totalcharges'].fillna(round(churnData['totalcharges'].mean(), 2), inplace=True)
churnData['totalcharges'].isnull().sum()

0

In [134]:
churnData['totalcharges']

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: totalcharges, Length: 7043, dtype: float64

## Build the LR model

- Use the following features: `tenure`, `SeniorCitizen`, `MonthlyCharges` and `TotalCharges`:
  - Scale the features either by using normalizer or a standard scaler.
  - Split the data into a training set and a test set.
  - Fit a logistic regression model on the training data.
  - Check the accuracy on the test data.

In [143]:
y = churnData['churn']
X = churnData[['tenure', 'seniorcitizen', 'monthlycharges', 'totalcharges']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [144]:
transformer = MinMaxScaler().fit(X_train) # need to keep transformer
X_train_normalized = transformer.transform(X_train)
X_train_norm = pd.DataFrame(X_train_normalized)
X_train_norm.columns = X_train.columns

In [145]:
transformer = MinMaxScaler().fit(X_test) # need to keep transformer
X_test_normalized = transformer.transform(X_test)
X_test_norm = pd.DataFrame(X_test_normalized)
X_test_norm.columns = X_test.columns

In [146]:
classification = LogisticRegression(random_state=0, solver='saga', multi_class='multinomial').fit(X_train_norm, y_train)

In [147]:
classification.score(X_train_norm, y_train)

0.7880724174653887

In [148]:
predictions = classification.predict(X_test_norm)
predictions

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [149]:
classification.score(X_test_norm, y_test)

0.8034066713981547

## Resample the data

Managing imbalance in the dataset
- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model is.


In [89]:
churnData['churn'].value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [151]:
category_0 = churnData[churnData['churn'] =='No']
category_1 = churnData[churnData['churn'] == 'Yes']

In [152]:
category_1_undersampled = resample(category_1, 
                                   replace=True, 
                                   n_samples = len(category_0))

In [153]:
churnData_oversampled = pd.concat([category_0, category_1_undersampled], axis=0)

In [154]:
churnData_oversampled['churn'].value_counts()

No     5174
Yes    5174
Name: churn, dtype: int64

## Build the LR model with resampled data

In [155]:
y = churnData_oversampled['churn']
X = churnData_oversampled[['tenure', 'seniorcitizen', 'monthlycharges', 'totalcharges']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [156]:
transformer = MinMaxScaler().fit(X_train) # need to keep transformer
X_train_normalized = transformer.transform(X_train)
X_train_norm = pd.DataFrame(X_train_normalized)
X_train_norm.columns = X_train.columns

transformer = MinMaxScaler().fit(X_test) # need to keep transformer
X_test_normalized = transformer.transform(X_test)
X_test_norm = pd.DataFrame(X_test_normalized)
X_test_norm.columns = X_test.columns

In [157]:
classification = LogisticRegression(random_state=0, solver='saga', multi_class='multinomial').fit(X_train_norm, y_train)
classification.score(X_train_norm, y_train)

0.7382217927035516

In [159]:
predictions = classification.predict(X_test_norm)

In [160]:
classification.score(X_test_norm, y_test)

0.7309178743961353